In [13]:
import scraping_packers_homepage.env as env
llm = env.llm

search_engine_id = ""
api_key = ""

In [7]:
import pandas as pd

df = pd.read_excel('/Users/yoonhae/Downloads/澳大利亚输华大麦小麦仓储企业名单.xlsx', header=2)
df.rename(columns={'Registered Establishment Number': "no", 
           "Registered Establishment Name": "comp_name",
           "City": "city",
           "State": "state"}, inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   no         100 non-null    int64 
 1   comp_name  100 non-null    object
 2   city       100 non-null    object
 3   state      100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.9+ KB


In [30]:
import json

def reduce_search_result(search_result):
    core_info_list = []

    core_col_list = {"link", "displayLink", "title", "snippet"}
    core_meta_tags = ['title', 'descript', 'name', 'type']
    exclude_keywords = ['twitter', "facebook"]

    for item in search_result["items"]:
        # pdf url 이면 작업하지 않는다. 
        if item['link'].endswith('.pdf'):
            continue

        core_info = {k: v for k, v in item.items() if k in core_col_list}
        metatags = []
        for tag_row in item.get('pagemap', {}).get('metatags', []):
            for key, value in tag_row.items():
                skip = False
                for tag in exclude_keywords:
                    if tag in key:
                        skip = True
                        break
                if skip:
                    continue
                
                filtered_tag_row = {}
                # meta tag 는 사이트 제작자마다 달라서 keyword가 포함되는지를 확인해보자.
                for tag in core_meta_tags:
                    if tag in key:
                        filtered_tag_row[key] = value
                        break
            metatags.append(filtered_tag_row)

        core_info["pagemap"] = {}
        core_info["pagemap"]["metatags"] = metatags
        core_info_list.append(core_info)
    
    return core_info_list

In [ ]:
from scraping_packers_homepage.llm_models.hompage_selector import select_comp_homepage_with_llm
from scraping_packers_homepage.tools.google_searcher import search

packer_list = ['AGP Grain Marketing, LLC',
                'AVERE COMMODITIES CORP',
               'A & R Logistics, Inc.',
                'ADM Grain Company', 
                'Affton A Dana Company', 
                'AGP Grain Ltd.', 
                'AGP Grain Marketing, LLC.', 
                'AGP Processing, Inc', 
                'Agroex International INC.', 
                'Agropex International INC.', 
                'Alabama Dept. Of Agriculture and Industries', 
                'American Trade Partnership, LLC',]

def get_homepage(packer):
    result = search(packer, search_engine_id, api_key)
    #reduced_result = [item for item in result['items'] if not item['link'].endswith('.pdf')]
    reduced_result = reduce_search_result(result)
    #url = select_comp_homepage_with_llm(llm, packer, result["items"][:5]).strip('" ')
    url = select_comp_homepage_with_llm(llm, packer, reduced_result[:5]).strip('" ')
    if not url:
        #url = select_comp_homepage_with_llm(llm, packer, result["items"][5:]).strip('" ')
        url = select_comp_homepage_with_llm(llm, packer, reduced_result[5:]).strip('" ')
    return url

# if True:
#     packer = packer_list[0]
#     print(get_homepage(packer))
df['homepage'] = df['comp_name'].apply(get_homepage)    

In [ ]:
tmp = search("AGP Grain Marketing, LLC", search_engine_id, api_key)
tmp

In [32]:
df.to_csv('./output/중국_20240202_0959.csv', index=False, quoting=1)

In [27]:
import json
print(json.dumps(result, ensure_ascii=False))

filtered_result = [item for item in result['items'] if not item['link'].endswith('.pdf')]

packer = "AGRIGRAIN (COONAMBLE) PTY LTD"
#url = select_comp_homepage_with_llm(llm, packer, result["items"][:5]).strip('" ')
#url = select_comp_homepage_with_llm(llm, packer, reduced_result[:5]).strip('" ')
url = select_comp_homepage_with_llm(llm, packer, filtered_result[:5]).strip('" ')
if not url:
    #url = select_comp_homepage_with_llm(llm, packer, result["items"][5:]).strip('" ')
    #url = select_comp_homepage_with_llm(llm, packer, reduced_result[5:]).strip('" ')
    url = select_comp_homepage_with_llm(llm, packer, filtered_result[5:]).strip('" ')
print(url)

{"kind": "customsearch#search", "url": {"type": "application/json", "template": "https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json"}, "queries": {"request": [{"title": "Google Custom Search - AGRIGRAIN (COONAMBLE) PTY LTD", "totalResults": "56", "searchTerms": "AGRIGRAIN (COONAMBLE) PTY LTD", "count": 10, "startIndex": 1, "inputEncoding": "utf8", "outputEncoding": "utf8", "safe": "off", "cx

In [14]:
url = select_comp_homepage_with_llm(llm, packer, reduced_result[:5])

url


In [40]:
import scraping_packers_homepage.llm_models.hompage_selector as hompage_selector
import importlib
importlib.reload(hompage_selector)

<module 'scraping_packers_homepage.llm_models.hompage_selector' from '/Users/yoonhae/test/crawling/scraping_packers_homepage/llm_models/hompage_selector.py'>

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry


response = requests.get('https://www.viterra.com/')
soup = BeautifulSoup(response.text, 'html.parser')

In [52]:
for meta in soup.find_all('meta'):
    print(meta.attrs)

print(soup.find('title'))

{'charset': 'utf-8'}
{'http-equiv': 'X-UA-Compatible', 'content': 'IE=edge'}
{'name': 'viewport', 'content': 'width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0'}
{'name': 'description', 'content': 'Empowering agriculture through connectivity. Our global network links producers and consumers for sustainable, traceable, quality-controlled products.'}
{'name': 'date', 'content': '2023-11-24T13:39:29.578Z'}
{'property': 'og:title', 'content': 'Viterra - The Agriculture Network'}
{'property': 'og:description', 'content': 'Empowering agriculture through connectivity. Our global network links producers and consumers for sustainable, traceable, quality-controlled products.'}
{'property': 'og:url', 'content': 'https://www.viterra.com'}
<title>Viterra - The Agriculture Network</title>
